# WineryPy
----

#### Note
* Instructions **

In [9]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
import gmaps
from scipy.stats import linregress
import scipy.stats as st
from pprint import pprint

# GooglePlace 
from GeoLoc import GooglePlaces
# Google developer API key
from config import gkey
#print(gkey)


In [10]:
name_list = []
address_list = []
rating_list = []
lat_list = []
lng_list = []

if __name__ == '__main__':
    api = GooglePlaces(gkey)
    places = api.search_places_by_text("wineries in Arizona, AZ", "restaurant,bar")
    #print(places)
    fields = ['name', 'formatted_address', 'rating', 'review'] 
    index_counter = 0
    for place in places:
        try:
            details = api.get_place_details(place['place_id'], fields)
            name_list.append(place["name"])
            address_list.append(place["formatted_address"])
            rating_list.append(place["rating"])
            lat_list.append(place["geometry"]["location"]["lat"])
            lng_list.append(place["geometry"]["location"]["lng"])
           
               
            #print(f"===================PLACE : ===================")
            #print("Name:", name_list)
            #print("Address:", address_list)
            #print("Rating:", rating_list)
            
            index_counter = index_counter + 1 
            
        except(KeyError, IndexError):
            print("winary not found. Skipping...")
        
        

### Convert Winery Raw Data to DataFrame
* Export the Winery data into a .csv.
* Display the DataFrame

In [11]:
# Create a panda data frame using data retrieved
winery_df = pd.DataFrame({ 
                "Name" : name_list,
                "Address" : address_list,
                "Rating" : rating_list,
                "Lat" : lat_list,
                "Lng" : lng_list
                
       
})

In [12]:
#  Count data
winery_df.count()

Name       60
Address    60
Rating     60
Lat        60
Lng        60
dtype: int64

In [13]:
# Display DataFrame
#winery_df.head()

In [14]:
# dropping null value columns to avoid errors 
winery_df.dropna(inplace = True) 

# new data frame with split address columns 
new_df = winery_df["Address"].str.split(",", n = 3, expand = True) 
  
# making separate Address column from new data frame 
winery_df["Address"]= new_df[0] 
  
# making separate city column from new data frame 
winery_df["City"]= new_df[1] 

# making separate State & Zipcode column from new data frame 
#brewery_df["State-Zip"]= new[2] 
winery_df[['State','Zipcode']] = new_df[2].str.split(expand=True) 

# making separate Country column from new data frame 
winery_df["Country"]= new_df[3] 
  
#df display 
winery_df.head()

,Name,Address,Rating,Lat,Lng,City,State,Zipcode,Country
0,Winery 101 Peoria,9299 W Olive Ave STE 101,4.8,33.565257,-112.259074,Peoria,AZ,85345,United States
1,Gallifant Cellars,9299 W Olive Ave #101,4.7,33.565263,-112.258997,Peoria,AZ,85345,United States
2,"Carlson Creek Vineyard, Scottsdale Tasting Room",4142 N Marshall Way,4.8,33.495609,-111.929291,Scottsdale,AZ,85251,United States
3,Su Vino Winery,7330 E Main St Suite 105,3.9,33.493326,-111.923837,Scottsdale,AZ,85251,United States
4,Aridus Wine Company Tasting Room,7173 E Main St,4.6,33.492828,-111.926724,Scottsdale,AZ,85251,United States


In [15]:
# Extract relevant fields from the data frame


# Export the City_Data into a csv
winery_df.to_csv("DataFile/winery.csv", index = False)

In [16]:
# Configure gmaps
gmaps.configure(api_key=gkey)

# Store latitude and longitude in locations
locations = winery_df[["Lat", "Lng"]]

# Fill NaN values and convert to float
rating = winery_df["Rating"].astype(float)

In [20]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=.1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [18]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Name}</dd>
<dt>Address</dt><dd>{Address}</dd><dd>{City}</dd>
<dt>Rating</dt><dd>{Rating}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
bar_info = [info_box_template.format(**row) for index, row in winery_df.iterrows()]
locations = winery_df[["Lat", "Lng"]]

In [19]:
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = bar_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))